<a href="https://colab.research.google.com/github/Cristina2003leon/master-analitica-datos-cristina-mantilla/blob/main/Search_grounding_for_research_report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nueva sección

In [ ]:
import sqlite3
import pandas as pd

# If 'con' exists from a previous run and is an open connection, close it defensively.
# This prevents potential locks from previous partial executions.
if 'con' in globals() and isinstance(con, sqlite3.Connection):
    try:
        con.close()
        print("Closed existing SQLite connection.")
    except Exception as e:
        print(f"Could not close previous connection: {e}")


!rm base_datos.db 2>/dev/null # Also remove the main db file itself to ensure a clean start

con = sqlite3.connect("base_datos.db", timeout=5)

# Set busy_timeout PRAGMA for better handling of concurrent access
con.execute("PRAGMA busy_timeout = 5000;") # Wait up to 5 seconds for a lock

# Borrar tabla si existe
con.execute("DROP TABLE IF EXISTS animales;")

# Crear tabla
con.execute("""
CREATE TABLE animales(
    id INTEGER PRIMARY KEY,
    Nombre TEXT,
    PeligrosidadVeneno INTEGER,
    Habitat TEXT,
    EsperanzaDeVida INTEGER
);
""")

Closed existing SQLite connection.


In [ ]:
con.executemany("""
INSERT INTO animales (id, Nombre, PeligrosidadVeneno, Habitat, EsperanzaDeVida)
VALUES (?, ?, ?, ?, ?)
""", [
    (1, 'Serpiente Taipán', 10, 'Bosques secos de Australia', 15),
    (2, 'Rana Dardo Dorada', 9, 'Selva tropical', 10),
    (3, 'Viuda Negra', 8, 'Zonas templadas', 3),
    (4, 'Mamba Negra', 9, 'Sabana africana', 20),
    (5, 'Escorpión Amarillo', 7, 'Regiones áridas', 6),
])

con.commit()


In [ ]:
pd.read_sql("SELECT * FROM animales;", con)


,id,Nombre,PeligrosidadVeneno,Habitat,EsperanzaDeVida
0,1,Serpiente Taipán,10,Bosques secos de Australia,15
1,2,Rana Dardo Dorada,9,Selva tropical,10
2,3,Viuda Negra,8,Zonas templadas,3
3,4,Mamba Negra,9,Sabana africana,20
4,5,Escorpión Amarillo,7,Regiones áridas,6


In [ ]:
con.execute("DROP TABLE IF EXISTS ecologia;")

con.execute("""
CREATE TABLE ecologia(
    id INTEGER,
    Dieta TEXT,
    Region TEXT,
    PoblacionEstimada INTEGER
);
""")


In [ ]:
con.executemany("""
INSERT INTO ecologia (id, Dieta, Region, PoblacionEstimada)
VALUES (?, ?, ?, ?)
""", [
    (1, 'Carnívoro', 'Australia', 100000),
    (2, 'Insectívoro', 'Colombia', 5000),
    (3, 'Insectívoro', 'Norteamérica / Europa', 200000),
    (4, 'Carnívoro', 'África', 50000),
    (5, 'Insectívoro', 'Oriente Medio', 800000),
])

con.commit()


In [ ]:
pd.read_sql("SELECT * FROM ecologia;", con)


,id,Dieta,Region,PoblacionEstimada
0,1,Carnívoro,Australia,100000
1,2,Insectívoro,Colombia,5000
2,3,Insectívoro,Norteamérica / Europa,200000
3,4,Carnívoro,África,50000
4,5,Insectívoro,Oriente Medio,800000


In [ ]:
query = """
SELECT
    a.Nombre,
    a.PeligrosidadVeneno,
    a.Habitat,
    a.EsperanzaDeVida,
    e.Dieta,
    e.Region,
    e.PoblacionEstimada
FROM animales a
JOIN ecologia e
    ON a.id = e.id;
"""

pd.read_sql(query, con)


,Nombre,PeligrosidadVeneno,Habitat,EsperanzaDeVida,Dieta,Region,PoblacionEstimada
0,Serpiente Taipán,10,Bosques secos de Australia,15,Carnívoro,Australia,100000
1,Rana Dardo Dorada,9,Selva tropical,10,Insectívoro,Colombia,5000
2,Viuda Negra,8,Zonas templadas,3,Insectívoro,Norteamérica / Europa,200000
3,Mamba Negra,9,Sabana africana,20,Carnívoro,África,50000
4,Escorpión Amarillo,7,Regiones áridas,6,Insectívoro,Oriente Medio,800000
